In [1]:
import torch

## torch.tensor 인수
#### data
- 배열
#### dtype
- 데이터를 저장할 자료형

1. 논리형

||||
|-|-|-|
| 논리형 8비트 | torch.bool | True, False |

2. 정수형

||||
|-|-|-|
| 정수형 8비트 | torch.int or torch.char | $-2^7$ ~ $2^7-1$ |
| 정수형 16비트 | torch.int16 or torch.short | $-2^15$ ~ $2^15-1$ |
| 정수형 32비트 | torch.int32 or torch.int | $-2^31$ ~ $2^31-1$ |
| 정수형 64비트 | torch.int64 or torch.long | $-2^63$ ~ $2^63-1$ |

3. 실수형

||||
|-|-|-|
| 실수형 16비트 | torch.float16 or torch.half | $±5.96×10^-8$ ~ $±6.55×10^4$ |
| 실수형 32비트 | torch.float32 or torch.float | $±1.40×10^-45$ ~ $±3.40×10^38$ |
| 실수형 64비트 | torch.float64 or torch.double | $±4.94×10^-324$ ~ $±1.79×10^308$ |

#### device
- 기기 설정(cpu, cpu)

#### requires_grad
- 텐서에 대한 기울기를 저장할지 결정

In [4]:
x_1 = torch.Tensor(2,3) #2x3 난수로 이루어진 파이토치 텐서 생성
x_2 = torch.tensor([[1,2,3],[4,5,6]]) #2x3 파이토치 텐서 생성
print(x_1)
print(x_2)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[1, 2, 3],
        [4, 5, 6]])


In [10]:
x_f = torch.tensor([[1,2,3],[4,5,6]], dtype=torch.float32) #float32 타입의 텐서 생성
x_d = torch.tensor([[1,2,3],[4,5,6]], dtype=torch.float64) #float64 타입의 텐서 생성
x_b = torch.tensor([[1,2,3],[4,5,6]], dtype=torch.int8) #uint8 타입의 텐서 생성
print(x_f)
print(x_d)
print(x_b)

tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor([[1., 2., 3.],
        [4., 5., 6.]], dtype=torch.float64)
tensor([[1, 2, 3],
        [4, 5, 6]], dtype=torch.int8)


In [33]:
x = torch.tensor(data=[2.0,3.0], requires_grad=True) #미분 가능 텐서 생성
y = x**2 #y=[4.0,9.0]
z = 2*y + 3 #z=[11.0,21.0]
target = torch.tensor([3.0,4.0])
loss = torch.sum(torch.abs(z-target)) #loss = |11-3| + |21-4| = 25.0
loss.backward()

print(x.grad, z.grad_fn) # z=x^2+3 의 미분값, 

tensor([ 8., 12.]) <AddBackward0 object at 0x000001DC27A952A0>


In [49]:
import torch
import torch.nn as nn # 신경망 모델(Linear)
import torch.optim as optim # 옵티마이저(경사하강법)
import torch.nn.init as init # 텐서에 초기값 주기

num_data = 1000
num_epoch = 500

# uniform: 균등
# -10~10까지 균등하게 [num_data,1]크기의 텐서 생성([-10~10 사이의 수]×1000개)
x = init.uniform_(torch.Tensor(num_data, 1), -10, 10)

# 실제 데이터는 균일하지 않기 때문에 현실성 반영을 위한 노이즈 생성
# torch.init.normal_: 정규분포 생성
# [num_data, 1] 크기의 평균 0, 표준편차 1인 텐서 생성
noise = init.normal_(torch.FloatTensor(num_data,1), std=1)

In [50]:
y = 2*x+3
y_noise = y + noise

In [51]:
# torch.nn.Linear(input, output)
# 입력과 출력의 특성이 1개
model = nn.Linear(1,1)

# 모델의 출력값과 y_noise의 오차를 계산하는 손실함수
# L1Loss: 평균 절대 오차
loss_func = nn.L1Loss()

# opimizer: 최적화 함수
# 여기서는 SGD(stochastic gradient descent) 사용
# SGD: 한 번에 들어오는 수대로 경사하강법을 적용
optimizer = optim.SGD(model.parameters(), lr=0.01)

## 선형 회귀 분석
- z = x × w + b의 직선 함수에서 w와 b를 찾는 과정
- 모델의 output과 label을 손실 함수에 넣고 그 결과가 작아지는 쪽으로 가중치와 편향을 업데이트

In [53]:
label = y_noise

for i in range(num_epoch):
    # zero_grad(): 기울기를 0으로 초기화(다음 epoch때 새로운 기울기를 구하기 위해)
    optimizer.zero_grad()
    output = model(x)

    loss = loss_func(output, label)
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print(f"epoch{i}: {loss.data}")
        param_list = list(model.parameters())
        print(f"weight: {param_list[0].data}, bias: {param_list[1].data}")


# parameters(): 모델의 파라미터(가중치, 편향)들을 반환
# parameters[0]: 가중치, parameters[1]: 편향 -> 2.0019, 2.9086 (2,3에 가까운 값)

#print(param_list[0].item(), param_list[1].item())

epoch0: 0.8373180031776428
weight: tensor([[1.9900]]), bias: tensor([2.8793])
epoch100: 0.835016131401062
weight: tensor([[1.9894]]), bias: tensor([2.9261])
epoch200: 0.834199845790863
weight: tensor([[1.9865]]), bias: tensor([2.9540])
epoch300: 0.8340947031974792
weight: tensor([[1.9855]]), bias: tensor([2.9633])
epoch400: 0.8340680003166199
weight: tensor([[1.9850]]), bias: tensor([2.9681])
